# Travailler avec les paires clefs-valeurs


## Chargement des données

In [10]:
num1 = sc.parallelize([1,1,1,2,3,4,5,5,6,7,8,8,8])
num2 = sc.parallelize(xrange(4,11))
txt1 = sc.parallelize(['allo les amis', 'ne riez pas', 'de mon apprentissage'])
txt2 = sc.textFile("/etc/passwd")
txt3 = sc.parallelize(["BRLAV35;Lavoie;Bruno","FRPOL9;Poliquin;Frederic","DOASS4;Asselin;Dominique","ALWHI3;Whittom;Allen"])

##  Créer des pairs-RDDs

In [40]:
# Créer une paire numéro --> valeur au carré
pair1 = num1.map(lambda x: (x, x*x))
pair1.collect()

[(1, 1),
 (1, 1),
 (1, 1),
 (2, 4),
 (3, 9),
 (4, 16),
 (5, 25),
 (5, 25),
 (6, 36),
 (7, 49),
 (8, 64),
 (8, 64),
 (8, 64)]

In [23]:
# Créer une paire du type code_utilisateur -> informations
pair3 = txt3.map(lambda x: (x.split(";", 1)[0], x.split(";", 1)[1]))
pair3.collect()

[('BRLAV35', 'Lavoie;Bruno'),
 ('FRPOL9', 'Poliquin;Frederic'),
 ('DOASS4', 'Asselin;Dominique'),
 ('ALWHI3', 'Whittom;Allen')]

## Transformations sur les pairs-RDDs

### reduceByKey(func)
__Utilité:__ Fusionne ensemble les valeurs de chaque clef en utilisant une fonction associative.

In [25]:
pair1.reduceByKey(lambda x,y: x + y).collect()

[(8, 192), (4, 16), (1, 3), (5, 50), (2, 4), (6, 36), (3, 9), (7, 49)]

In [34]:
# On peut voir que la fonction ne peut pas être appelée s'il y a seulement qu'un élément pour une clef.
pair1.reduceByKey(lambda x,y: x>0).collect()

[(8, True), (4, 16), (1, True), (5, True), (2, 4), (6, 36), (3, 9), (7, 49)]

### groupByKey()
__Utilité:__ Groupe les valeurs qui ont la même clef. Ressemble drôlement un à un GROUP BY + COLLECT en SQL.

In [41]:
# retourne un itérable...
pair1.groupByKey().collect()

[(8, <pyspark.resultiterable.ResultIterable at 0x7f1b943bd790>),
 (4, <pyspark.resultiterable.ResultIterable at 0x7f1b943a6090>),
 (1, <pyspark.resultiterable.ResultIterable at 0x7f1b943a65d0>),
 (5, <pyspark.resultiterable.ResultIterable at 0x7f1b943a62d0>),
 (2, <pyspark.resultiterable.ResultIterable at 0x7f1b943a64d0>),
 (6, <pyspark.resultiterable.ResultIterable at 0x7f1b943a6550>),
 (3, <pyspark.resultiterable.ResultIterable at 0x7f1b943a6450>),
 (7, <pyspark.resultiterable.ResultIterable at 0x7f1b943a6410>)]

In [39]:
# petit truc pour visualiser, on fait un map sur les valeurs (mapValues)...
pair1.groupByKey().mapValues(list).collect()

[(8, [64, 64, 64]),
 (4, [16]),
 (1, [1, 1, 1]),
 (5, [25, 25]),
 (2, [4]),
 (6, [36]),
 (3, [9]),
 (7, [49])]

### combineByKey(createCombiner, mergeValue, mergeCombiners)
__Utilité:__ Combine les valeurs ayant la même clef, en retournant un type différent.

Generic function to combine the elements for each key using a custom set of aggregation functions.

Turns an RDD[(K, V)] into a result of type RDD[(K, C)], for a “combined type” C. Note that V and C can be different – for example, one might group an RDD of type (Int, Int) into an RDD of type (Int, List[Int]).

Users provide three functions:

- __createCombiner__, which turns a V into a C (e.g., creates a one-element list)
- __mergeValue__, to merge a V into a C (e.g., adds it to the end of a list)
- __mergeCombiners__, to combine two C’s into a single one.

Lien: http://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=sample#pyspark.RDD.combineByKey

In [44]:
pair1.collect()

[(1, 1),
 (1, 1),
 (1, 1),
 (2, 4),
 (3, 9),
 (4, 16),
 (5, 25),
 (5, 25),
 (6, 36),
 (7, 49),
 (8, 64),
 (8, 64),
 (8, 64)]

In [107]:
# Convertir le type comme c'est réellement l'intention de la fonction
# Aucun + ne devrait apparaitre en local si nous n'avons pas plus d'une partitions.

pair1.combineByKey(lambda x: str(x), 
                   lambda x,y: x + '*' + str(y), 
                   lambda x,y: x + '+' + y).collect()


[(8, '64*64*64'),
 (4, '16'),
 (1, '1*1*1'),
 (5, '25*25'),
 (2, '4'),
 (6, '36'),
 (3, '9'),
 (7, '49')]

In [119]:
# On peut émuler un groupByKey

pair1.combineByKey(lambda v:     [v], 
                   lambda c,v:   c + [v], 
                   lambda c1,c2: c1 + c2).collect()


[(8, [64, 64, 64]),
 (4, [16]),
 (1, [1, 1, 1]),
 (5, [25, 25]),
 (2, [4]),
 (6, [36]),
 (3, [9]),
 (7, [49])]

### mapValues(func)
__Utilité:__ appliquer une fonction à chaque valeur sans changer la clef.

In [124]:
pair1.mapValues(lambda x: 'x' + str(x*2)).collect()

[(1, 'x2'),
 (1, 'x2'),
 (1, 'x2'),
 (2, 'x8'),
 (3, 'x18'),
 (4, 'x32'),
 (5, 'x50'),
 (5, 'x50'),
 (6, 'x72'),
 (7, 'x98'),
 (8, 'x128'),
 (8, 'x128'),
 (8, 'x128')]

### flatMapValues(func)
__Utilité:__ appliquer une fonction qui retourne un itérateur pour chaque valeur d'un pair-RDD et chaque élément retourné produit une entrée clef-valeur avec la vieille clef. Souvent utilisé pour tokenization.

In [127]:
pair3.collect()

[('BRLAV35', 'Lavoie;Bruno'),
 ('FRPOL9', 'Poliquin;Frederic'),
 ('DOASS4', 'Asselin;Dominique'),
 ('ALWHI3', 'Whittom;Allen')]

In [130]:
pair3.flatMapValues(lambda x: x.split(';')).collect()

[('BRLAV35', 'Lavoie'),
 ('BRLAV35', 'Bruno'),
 ('FRPOL9', 'Poliquin'),
 ('FRPOL9', 'Frederic'),
 ('DOASS4', 'Asselin'),
 ('DOASS4', 'Dominique'),
 ('ALWHI3', 'Whittom'),
 ('ALWHI3', 'Allen')]

In [134]:
# Peut aussi être utile pour pivoter les valeurs qui sont des listes
# Créer les listes en valeurs...
temp = pair3.mapValues(lambda x: x.split(';'))
temp.collect()

[('BRLAV35', ['Lavoie', 'Bruno']),
 ('FRPOL9', ['Poliquin', 'Frederic']),
 ('DOASS4', ['Asselin', 'Dominique']),
 ('ALWHI3', ['Whittom', 'Allen'])]

In [136]:
temp.flatMapValues(lambda x: x).collect()

[('BRLAV35', 'Lavoie'),
 ('BRLAV35', 'Bruno'),
 ('FRPOL9', 'Poliquin'),
 ('FRPOL9', 'Frederic'),
 ('DOASS4', 'Asselin'),
 ('DOASS4', 'Dominique'),
 ('ALWHI3', 'Whittom'),
 ('ALWHI3', 'Allen')]